In [1]:
import pandas as pd
from alpha_vantage.timeseries import TimeSeries
import datetime

In [2]:
ts = TimeSeries(key = 'API_KEY_HERE')

In [3]:
interval = '5min'
date = '2023-1-01'

In [4]:
class ScriptData:
    
    def __init__(self, script):
        self.script = script
        self.fetch_intraday_data(self.script)
        self.df = self.convert_intraday_data(self.data)

    def fetch_intraday_data(self, script):
        self.data, meta_data = ts.get_intraday(script)
        return self.data

    def fetch_historical_data(self, script):
        data_01, meta_data_01 = ts.get_intraday(symbol=script, interval=interval, outputsize='full')
        self.data_list_01 = []
        for timestamp, items in data_01.items():
            open_item = float(items["1. open"])
            high_item = float(items["2. high"])
            low_item = float(items["3. low"])
            close_item = float(items["4. close"])
            volume = int(items["5. volume"])
            self.data_list_01.append([timestamp, open_item, high_item, low_item, close_item, volume])
        df_02 = pd.DataFrame(self.data_list_01, columns=["timestamp", "open", "high", "low", "close", "volume"])
        df_02["timestamp"] = pd.to_datetime(df_02["timestamp"])
        return df_02

    def convert_intraday_data(self, data):
        self.data_list = []
        for timestamp, items in data.items():
            open_item = float(items["1. open"])
            high_item = float(items["2. high"])
            low_item = float(items["3. low"])
            close_item = float(items["4. close"])
            volume = int(items["5. volume"])
            self.data_list.append([timestamp, open_item, high_item, low_item, close_item, volume])
        df = pd.DataFrame(self.data_list, columns=["timestamp", "open", "high", "low", "close", "volume"])
        df["timestamp"] = pd.to_datetime(df["timestamp"])
        return df

    def indicator1(self, timestamp, dataf, timeperiod = 5):
        indicator_data = []
        final_output = {"timestamp" : [], "indicator" : []}
        upcount = 0
        downcount = timeperiod - 1
        while downcount < len(dataf):
            cal_temp = 0
            for i in range(upcount, downcount + 1):
                cal_temp = float(dataf["close"][i]) + cal_temp
            final_temp = cal_temp/timeperiod
            indicator_data.append(final_temp)
            if downcount + 1 >= len(dataf):
                break
            upcount += 1
            downcount += 1
        for i in range(0, len(dataf)):
            final_output["timestamp"].append(dataf["timestamp"][i])
        for i in range(0, len(dataf)):
            if i < timeperiod - 1:
                final_output["indicator"].append("NaN")
            else:
                final_output["indicator"].append(indicator_data[i-timeperiod + 1])
        df_1 = pd.DataFrame(final_output)
        df_1.fillna(value = pd.np.nan, inplace = True)
        return df_1

    def __getitem__(self, index):
        return self.df.loc[index]

    def __setitem__(self, index, value):
        self.df.iloc[index] = value

    def __contains__(self, value):
        return self.script == value

In [5]:
script_data = ScriptData("GOOGL")

In [6]:
script_data.convert_intraday_data(script_data.fetch_intraday_data("GOOGL"))

,timestamp,open,high,low,close,volume
0,2023-02-14 20:00:00,94.2400,94.24,93.500,94.1200,27980
1,2023-02-14 19:45:00,94.2700,94.32,94.200,94.2000,11286
2,2023-02-14 19:30:00,94.3400,94.34,94.250,94.3000,2459
3,2023-02-14 19:15:00,94.4500,94.50,94.260,94.2600,13234
4,2023-02-14 19:00:00,94.3000,94.49,94.300,94.3600,14382
...,...,...,...,...,...,...
95,2023-02-13 12:15:00,94.2900,94.43,94.220,94.3950,1182749
96,2023-02-13 12:00:00,94.3064,94.36,94.120,94.3000,1523977
97,2023-02-13 11:45:00,94.4600,94.50,93.980,94.3021,2155180
98,2023-02-13 11:30:00,94.6500,94.80,94.380,94.4500,1425631


In [7]:
script_data.df

,timestamp,open,high,low,close,volume
0,2023-02-14 20:00:00,94.2400,94.24,93.500,94.1200,27980
1,2023-02-14 19:45:00,94.2700,94.32,94.200,94.2000,11286
2,2023-02-14 19:30:00,94.3400,94.34,94.250,94.3000,2459
3,2023-02-14 19:15:00,94.4500,94.50,94.260,94.2600,13234
4,2023-02-14 19:00:00,94.3000,94.49,94.300,94.3600,14382
...,...,...,...,...,...,...
95,2023-02-13 12:15:00,94.2900,94.43,94.220,94.3950,1182749
96,2023-02-13 12:00:00,94.3064,94.36,94.120,94.3000,1523977
97,2023-02-13 11:45:00,94.4600,94.50,93.980,94.3021,2155180
98,2023-02-13 11:30:00,94.6500,94.80,94.380,94.4500,1425631


In [8]:
"GOOGL" in script_data

True

In [9]:
script_data = ScriptData("AAPL")
script_data.convert_intraday_data(script_data.fetch_intraday_data("AAPL"))

,timestamp,open,high,low,close,volume
0,2023-02-14 20:00:00,152.750,152.850,152.7000,152.8400,17795
1,2023-02-14 19:45:00,153.010,153.020,152.7500,152.7600,15033
2,2023-02-14 19:30:00,152.850,153.030,152.8200,153.0300,10460
3,2023-02-14 19:15:00,153.080,153.090,152.8400,152.8400,9621
4,2023-02-14 19:00:00,153.080,153.090,153.0700,153.0900,1409
...,...,...,...,...,...,...
95,2023-02-13 12:15:00,153.440,153.878,153.4100,153.8300,1507638
96,2023-02-13 12:00:00,153.510,153.690,153.2800,153.4200,1741812
97,2023-02-13 11:45:00,153.150,153.520,152.9678,153.5100,1815044
98,2023-02-13 11:30:00,152.894,153.210,152.8940,153.1423,2038315


In [10]:
script_data.df

,timestamp,open,high,low,close,volume
0,2023-02-14 20:00:00,152.750,152.850,152.7000,152.8400,17795
1,2023-02-14 19:45:00,153.010,153.020,152.7500,152.7600,15033
2,2023-02-14 19:30:00,152.850,153.030,152.8200,153.0300,10460
3,2023-02-14 19:15:00,153.080,153.090,152.8400,152.8400,9621
4,2023-02-14 19:00:00,153.080,153.090,153.0700,153.0900,1409
...,...,...,...,...,...,...
95,2023-02-13 12:15:00,153.440,153.878,153.4100,153.8300,1507638
96,2023-02-13 12:00:00,153.510,153.690,153.2800,153.4200,1741812
97,2023-02-13 11:45:00,153.150,153.520,152.9678,153.5100,1815044
98,2023-02-13 11:30:00,152.894,153.210,152.8940,153.1423,2038315


In [11]:
"AAPL" in script_data

True

In [12]:
class Strategy:

    def __init__(self, script_data):
        self.script_data = script_data

    def fetch_intraday_historical_day_data(self, symbol):
        a = self.script_data.fetch_historical_data(symbol)
        close_values = [i for i in a["close"]]
        b = self.script_data.indicator1(a["timestamp"], a)
        b = b.rename(columns={'indicator': 'indicator_data'})
        for i in range(0, len(a["close"])):
            b["close_data"] = close_values
        return b

    def get_signal(self, symbol):
        a = self.script_data.fetch_historical_data(symbol)
        close_values = [i for i in a["close"]]
        b = self.script_data.indicator1(a["timestamp"], a)
        b = b.rename(columns={'indicator': 'indicator_data'})
        for i in range(0, len(a["close"])):
            b["close_data"] = close_values
        c = {"timestamp" : [], "signal" : []}
        for i in range(0, len(b)):
            if float(b["indicator_data"].iloc[i]) < float(b["close_data"].iloc[i]):
                c["timestamp"].append(b["timestamp"][i])
                c["signal"].append("SELL")
            elif float(b["indicator_data"].iloc[i]) > float(b["close_data"].iloc[i]):
                c["timestamp"].append(b["timestamp"][i])
                c["signal"].append("BUY")
            else:
                c["timestamp"].append(b["timestamp"][i])
                c["signal"].append("NO SIGNAL")
        df_03 = pd.DataFrame(c)
        df_03 = df_03[df_03['signal'] != 'NO SIGNAL'].reset_index(drop=True)
#         df_03.to_excel('output.xlsx', index=True)
        return df_03

In [13]:
strategy_data = Strategy(script_data)

In [14]:
strategy_data.get_signal("GOOGL")

C:\Users\akkin\AppData\Local\Temp\ipykernel_14292\935254902.py:64: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead.
  df_1.fillna(value = pd.np.nan, inplace = True)


,timestamp,signal
0,2023-02-14 19:40:00,SELL
1,2023-02-14 19:35:00,SELL
2,2023-02-14 19:30:00,SELL
3,2023-02-14 19:20:00,BUY
4,2023-02-14 19:15:00,BUY
...,...,...
3832,2023-01-17 04:25:00,SELL
3833,2023-01-17 04:20:00,SELL
3834,2023-01-17 04:15:00,SELL
3835,2023-01-17 04:10:00,SELL
